# Economics of Antibiotic Discovery

This code runs a game using a dataframe of probabilities of a trial progressing through each stage of clinical trials. 
It comes from Okhravi, C. (2020). Economics of Public Antibiotics Development. Frontiers in Public Health, 8.
Available from: http://dx.doi.org/10.3389/fpubh.2020.0016

![alt text](fpubh-08-00161-t001.jpg "Estimated costs and transition probabilities through Clinical Trials.")

 The disease and conditions are: acute bacterial otitis media (ABOM), acute bacterial skin and skin structure infections (ABSSSI), community acquired bacterial pneumonia (CABP), complicated intra-abdominal infections (CIAI), complicated urinary tract infections (CUTI), and hospital acquired/ventilator associated bacterial pneumonia (HABP/VABP).Times are in months and costs in millions (USD).
        
## Running the code

- Click the "Next Trial" Button to run a trial

- The probabilities and costs use median costs and probabilities  for acute bacterial otitis media (ABOM)

- The random probabilities for each stage are given, with the transition probabilities to succeed to the next stage. Costs are accumulated (in millions USD) until you get a drug that succeeds through all stages.

Try a number of runs to find your minimum spend.


In [1]:
import pandas as pd
import random
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Button, HBox, VBox
from IPython.display import HTML

pd.options.display.float_format = "{:,.2f}".format
trans_prob = pd.read_csv("TransitionProbabilities.csv") # Data frame to hold transition probailites and costs for each phase of piprline


Money_spent=0 # Total spend
trial_no=0 # Trial Number
success=False # Global boolean to catch success of a trial

def Money_update(add): # Add money from this trial to the total spent
    global Money_spent # Total money spent.

    y=Money_spent+add
    Money_spent=y
    return(y)
def trial(): # Function running a trial. returns Money_spent_this_trial, boolean if true that all four successful, dataframe with results
    global trans_prob # Calculated probabilitie sthis trial
    trial_prob=[None] *5  # Will hold probabilities to pass each stage of pipleline
    trial_result=[None] *5  # Will hold probabilities to pass each stage of pipleline
    for j in range(5): # Calculate probabilitirs this trial
        trial_prob[j]=random.randint(0,1000)/1000 # set up trial probabilities - 0-1 in 3 dp
    trans_prob['Prob']=trial_prob # add as new column to dataframe
    continue_attempt=True
    Money_spent_this_trial=0

    for i in range(len(trial_prob)): # Loop over probabilities this trial
        if(continue_attempt): # Only check probability if pass pevious stage.
            if(trial_prob[i]>trans_prob['Prob Trans'].iloc[i]):   # If prob this stage is > transition prob, STOP trial
                Money_spent_this_trial+=trans_prob['Cost'].iloc[i] # Add money already spent at this stage.
                continue_attempt=False # Signal that trial has stopped,
                trial_result[i]='STOP' # Hold in trial results that STOP
            else:  # Pass this stage.
                Money_spent_this_trial+=trans_prob['Cost'].iloc[i] # Add in cost of this stage,
                trial_result[i]='CONTINUE' # Put in trial_ result that CONTINUE to next.
    return Money_spent_this_trial,continue_attempt,trial_result # Return Money Spent this trial,continue attempt failed by end or not, trial_results outcome at each stage.

style = {'description_width': 'initial'} #Style for labels for Text boxes 
output_money = widgets.Text(description='Total Spend (Millions) :',layout = widgets.Layout(width='220px'),style=style)
output_trial = widgets.Text(description='Trial No :',layout = widgets.Layout(width='100px'),style=style)
output_success = widgets.Text(description='Success :',layout = widgets.Layout(width='120px'),style=style)
btn = Button(description='New Trial')
btn_restart = Button(description='Restart')
btn_blank=Button(description="")

left_box = VBox([btn,output_trial])
middle_box = VBox([btn_restart,output_success])
out = widgets.Output()
right_box=VBox([btn_blank,output_money,out])


def setup_ui(df): # Update the dataframe output
    global out
    with out:
        display(df,clear=True)
    return out
def btn_eventhandler(obj): # Button even handler for "New Trial"
    global success
    global Money_spent
    global trial_no
    global trans_prob
    if not success: # Next Trial - if we have not been successful so far.
        trial_no+=1  # Incerement trial number
        return_values=trial() # Run trial, use return_values as wrapper for resulys
        x=float(return_values[0])# Money_spent
#
        for i in range(len(return_values[2])-1): # Loop over the trial results (STOP,CONTINUE). If fail earlier, set all subsequent to STOP as well.
            if(return_values[2][i]=="STOP"):
                return_values[2][i+1]="STOP"
        trans_prob['Result']=return_values[2]
#
        z=round(Money_update(x),2) # Total Money updated with money spent this trial
        output_trial.value=str(trial_no) # Values to output in Text Box fro Trial number.
#        
        if(return_values[1]): # If continue_attempt is true, we have had a successful trial. Set global success to true.
            output_success.value="YES" # Value for success textbox
            success=True
        else: # TRial not successful
            output_success.value="NO"
#            
        output_money.value=str(round(Money_spent,2)) # Output to textBox total money spent.
#    
        result_box = setup_ui(trans_prob[['Phase','Cost','Prob Trans','Prob','Result']].style.hide_index()) # Output data frame 
btn.on_click(btn_eventhandler)
def btn_restart_eventhandler(obj): # restart - reset variable for another series of trials
    global success
    global Money_spent
    global trial_no
    success=False
    Money_spent=0
    trial_no=0
    output_success.value="NO"
    output_trial.value=""
    output_money.value=""
    
btn_restart.on_click(btn_restart_eventhandler)

HBox([left_box, middle_box,right_box])# Set up layout of widgets.



/tmp/ipykernel_106/4235680167.py:88: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  result_box = setup_ui(trans_prob[['Phase','Cost','Prob Trans','Prob','Result']].style.hide_index()) # Output data frame
/tmp/ipykernel_106/4235680167.py:88: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  result_box = setup_ui(trans_prob[['Phase','Cost','Prob Trans','Prob','Result']].style.hide_index()) # Output data frame
/tmp/ipykernel_106/4235680167.py:88: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  result_box = setup_ui(trans_prob[['Phase','Cost','Prob Trans','Prob','Result']].style.hide_index()) # Output data frame
/tmp/ipykernel_106/4235680167.py:88: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  result_box = setup_ui(trans_prob[['Phase','Cost','Prob Trans','Prob','Result']].style.hide_index()) # Output data frame
/tmp/ipykernel_106/4235680167.py:88:

In [21]:
import pandas as pd
import ipywidgets as widgets

def button_run_on_click(_):
    status_label.value = "running...."

    df = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]])
    status_label.value = ""

    result_box = setup_ui(df)

    main_box.children = [button_run, status_label, result_box]


def setup_ui(df):
    out = widgets.Output()
    with out:
        display(df)
    return out


selection_box = widgets.Box()
selection_toggles = []
selected_labels = {}
default_labels = ['test1', "test2"]

labels = {"test1": "test1", "test2": "test2", "test3": "test3"}

def update_selection(change):
    owner = change['owner']
    name = owner.description

    if change['new']:
        owner.icon = 'check'
        selected_labels[name] = labels[name]
    else:
        owner.icon = ""
        selected_labels.pop(name)

for k in sorted(labels):
    o = widgets.ToggleButton(description=k)
    o.observe(update_selection, 'value')
    o.value = k in default_labels
    selection_toggles.append(o)
    selection_box.children = selection_toggles

status_label = widgets.Label()
status_label.layout.width = '300px'

button_run = widgets.Button(description="Run")
main_box = widgets.VBox([selection_box, button_run, status_label])
button_run.on_click(button_run_on_click)
display(main_box)


In [106]:
from ipywidgets import Button, HBox, VBox

words = ['correct', 'horse', 'battery', 'staple']
items = [Button(description=w) for w in words]
btn = Button(description='New Trial')
btn_restart = Button(description='Restart')
print(items[0])
left_box = VBox([btn])
right_box = VBox([btn_restart])


def btn_eventhandler(obj):
    global success
    global Money_spent
    global trial_no
    if not success:
        trial_no+=1
        print("\nTrial No "+str(trial_no))
        return_values=trial()
        x=float(return_values[0])
    
#    print("Type of x ",type(x))

        print( f"Money Spent this trial (millions), {x}.")
        print( f"Money Spent overall (millions), {round(Money_update(x),2)}.")
        output_trial.value=str(trial_no)
        if(return_values[1]):
            output_success.value="YES"
            success=True
        else:
            output_success.value="NO"
        output_money.value=str(round(Money_spent,2))
#btn.on_click(btn_eventhandler)
def btn_restart_eventhandler(obj):
    global success
    global Money_spent
    global trial_no
    success=False
    Money_spent=0
    trial_no=0
    output_success.value="NO"
    output_trial.value=""
    output_money.value=""
    
#btn_restart.on_click(btn_restart_eventhandler)


Button(description='correct', style=ButtonStyle())


In [33]:
from IPython.display import display
button = widgets.Button(description="Click Me!")
Money_spent =100
output = widgets.Text(
    value=str(Money_spent),
    description='Money :',
)

display(button, output)

def on_button_clicked(b):
    output.value="200"

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Text(value='100', description='Money :')

In [ ]:
trial_prob=[]